In [1]:
#fix weird branch stuff
#download libraries
%pip install Pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [31]:
#1: Get training data

#import images
from PIL import Image
from pathlib import Path

#count files in a folder
def count_files_in_folder(folder_path):
    folder = Path(folder_path)
    return sum(1 for item in folder.iterdir() if item.is_file())

#set folder path to l_training data
folder_path = 'l_training_data'
num_files = count_files_in_folder(folder_path)

imagelib = []

#store all files into library to use for data training
for i in range(num_files):
    imagelib.append("l_training_data/img" + str(i + 1) +".png")

#verify correct number of files
print(len(imagelib))

#open image with line below
#image = Image.open(filename)

2
